In [1]:
const R = require('ramda')

undefined

In [4]:
const Container = function(x) {
    this.__value = x;
}

Container.of = function(x) {
    return new Container(x);
}

[Function]

In [5]:
Container.of(3)

Container { __value: 3 }

In [6]:
Container.of('hotdogs')

Container { __value: 'hotdogs' }

In [7]:
Container.of(Container.of({
    name: 'yoda'
}))

Container { __value: Container { __value: { name: 'yoda' } } }

In [8]:
// (a -> b) -> Container a -> Container b
Container.prototype.map = function(f) {
    return Container.of(f(this.__value));
}

[Function]

In [9]:
Container.of(2).map(two => two + 2)

Container { __value: 4 }

In [11]:
Container.of('flamethrowers').map(s => s.toUpperCase())

Container { __value: 'FLAMETHROWERS' }

In [12]:
Container.of('bombs').map(R.concat(' away')).map(R.prop('length'))

Container { __value: 10 }

> ## Functors
> A `functor` is a type that implements map and obeys some laws. _(as does our container so far)_

> Which laws, Professor? 🤔

In [2]:
const Maybe = function(x) {
    this.__value = x
}

Maybe.of = function(x) {
    return new Maybe(x);
}

Maybe.prototype.isNothing = function() {
    return (this.__value === null || this.__value  === undefined);
}

Maybe.prototype.map = function(f) {
    return this.isNothing() ? Maybe.of(null) : Maybe.of(f(this.__value));
}

[Function]

> ### Maybe
> Like Container but skips over null or undefined values.

In [15]:
Maybe.of('Malkovich Malkovich').map(R.match(/a/ig))

Maybe { __value: null }

In [16]:
Maybe.of(null).map(R.match(/a/ig))

Maybe { __value: null }

In [17]:
Maybe.of({
    name: 'Boris'
}).map(R.prop('age')).map(R.add(10))

Maybe { __value: null }

In [18]:
Maybe.of({
    name: 'Dinah',
    age: 14
}).map(R.prop('age')).map(R.add(10))

Maybe { __value: null }

> _Expected Maybe(24)!_ 😰

In [19]:
Maybe.of({ name: 'Dinah', age: 14})

Maybe { __value: { name: 'Dinah', age: 14 } }

In [20]:
Maybe.of({ name: 'Dinah', age: 14}).map(R.prop('age'))

Maybe { __value: null }

In [3]:
// Correct a typo in isNothing
Maybe.prototype.isNothing = function() {
    return (this.__value === null || this.__value  === undefined);
}

[Function]

In [23]:
Maybe.of({ name: 'Dinah', age: 14}).map(R.prop('age'))

Maybe { __value: 14 }

> Yay!

In [24]:
Maybe.of({
    name: 'Dinah',
    age: 14
}).map(R.prop('age')).map(R.add(10))

Maybe { __value: 24 }

In [4]:
// map :: Functor f => (a -> b) -> fa -> fb
const map = R.curry((f, any_functor) => any_functor.map(f))

undefined

In [ ]:
map()

## Use Cases for Maybe
- In functions that may fail to return result.

In [5]:
const safeHead = xs => Maybe.of(xs[0])
const streetName = R.compose(map(R.prop('street')), safeHead, R.prop('addresses'))

undefined

In [6]:
streetName({ addresses: [] })

Maybe { __value: null }

In [9]:
streetName({
    addresses: [{
        street: 'Shady Ln',
        number: 4201
    }]
})

Maybe { __value: 'Shady Ln' }

## Pure Error Handling

In [2]:
const Left = function(x) {
    this.__value = x;
}

Left.of = function(x) {
    return new Left(x);
}

Left.prototype.map = function(f) {
    return this;
}

[Function]

In [3]:
const Right = function(x) {
    this.__value = x;
}

Right.of = function(x) {
    this.__value = x;
}

Right.prototype.map = function(f) {
    return Right.of(f(this.__value));
}

[Function]

In [14]:
Right.of('rain').map(str => 'b' + str)

TypeError: Cannot read property 'map' of undefined

In [4]:
Right.of = function(x) {
    return new Right(x);
}

[Function]

In [5]:
Right.of('rain').map(str => 'b' + str)

Right { __value: 'brain' }

In [6]:
Left.of('rain').map(str => 'b' + str)

Left { __value: 'rain' }

In [7]:
Right.of({
    host: 'localhost',
    port: 80
}).map(R.prop('host'))

Right { __value: 'localhost' }

In [8]:
Left.of({
    host: 'localhost',
    port: 80
}).map(R.prop('host'))

Left { __value: { host: 'localhost', port: 80 } }

> ### Little `maybe`
> You can use little `maybe` to get the value from a `Maybe`.

> Takes in a static value, a function and a `Maybe`. Returns the value in the `Maybe` if one exists else returns the static value.

```javascript
const maybe = R.curry((x, f, m) => m.isNothing() ? x : f(m.__value)))
```

> ### Either
> Using `Either(Left, Right)`, we can branch out in the code to signal failure and we'd know what caused the failure.

> Since `Left` ignores map requests, the original error message will not be changed till it gets to its destination.

In [5]:
const moment = require('moment')

// getAge :: Date -> User -> Either(String, Number)
const getAge = R.curry((now, user) => {
    const birthdate = moment(user.birthdate, 'YYYY-MM-DD');
    if (!birthdate.isValid()) {
        return Left.of('Birth date could not be parsed.');
    }
    return Right.of(now.diff(birthdate, 'years'));
});


undefined

In [10]:
getAge(moment(), {
    birthdate: '2006-04-26'
})

Right { __value: 10 }

In [14]:
getAge(moment(), {
    birthdate: '20011304'
})

Left { __value: 'Birth date could not be parsed.' }

In [10]:
//

undefined

In [7]:
// fortune :: Number -> String
const fortune = R.compose(R.concat('If you survive, you will be'), R.add(1))

// zoltar :: User -> Either(String, _)
const zoltar = R.compose(R.map(console.log), R.map(fortune), getAge(moment()))

undefined

In [8]:
zoltar({
    birthdate: '2006-04-2026'
})

If you survive, you will be11


Right { __value: undefined }

In [12]:
zoltar({
    birthdate: "Yay!"
})

Left { __value: 'Birth date could not be parsed.' }

> ### Little `either`
> Takes two functions and a static value. Each of those functions should return the same value.

In [11]:
// either :: (a -> c) -> (b -> c) -> Either(a b -> c)
const either = R.curry((f, g, e) => {
    switch(e.constructor) {
        case Left:
            return f(e.__value)
        case Right:
            return g(e.__value)
    }
});

undefined

In [16]:
const zolZoltar = R.compose(console.log, either(R.identity, fortune), getAge(moment()))

undefined

In [17]:
zolZoltar({
    birthdate: '2006-04-26'
})

If you survive, you will be11


undefined

In [18]:
zolZoltar({
    birthdate: 'Yay'
})

Birth date could not be parsed.


undefined

## From impure to Pure
- A function with side-effects can be made pure by wrapping its action in another function

```javascript
// getFromLocalStorage :: String -> (_ -> String)
const getFromLocalStorage = function(key) {
    return function() {
        return localStorage[key];
    }
}
```
`getLocalStorage` will now always return a function that, when called, will retrieve `key` from localStorage.

We can have a functor (`IO`) that will wrap over functions with side-effects and make them pure.

Unlike other functors (Maybe, Either, Container), `IO`'s value is always a function.

```javascript
const IO = function(f) {
    this.__value = f;
}

IO.of = function(x) {
    return new IO(function() {
        return x;
    });
};

IO.prototype.map = function(f) {
    return new IO(R.compose(f, this.__value))
}


//  io_window :: IO Window
var io_window = new IO(function() {
  return window;
});

io_window.map(function(win) {
  return win.innerWidth;
});
// IO(1430)

io_window.map(R.prop('location')).map(R.prop('href')).map(R.split('/'));
// IO(["http:", "", "localhost:8000", "blog", "posts"])


//  $ :: String -> IO [DOM]
var $ = function(selector) {
  return new IO(function() {
    return document.querySelectorAll(selector);
  });
};

$('#myDiv').map(R.head).map(function(div) {
  return div.innerHTML;
});
// IO('I am some inner html')
```

-----

Mapping over `IO` builds up an impure computation that has to be executed at some point. The responsibility of actually running the effects is delegated to the calling code.